## 1- Definition of arguments for function usage

In [1]:
import sys
import torch
import torch.optim as optim
import torchvision
from torchvision import transforms
from utils import *
import argparse
sys.argv = ['']

parser = argparse.ArgumentParser(description='Parameters training')
parser.add_argument('--model_architecture', type=str, default="VGG16", help='....')
parser.add_argument('--dataset', type=str, default="CIFAR10", help='....')
parser.add_argument('--batch_size', type=int, default=8, help='....')
parser.add_argument('--num_epochs', type=int, default=40, help='....')
parser.add_argument('--learning_rate', type=float, default=1e-3, help='....')
parser.add_argument('--optimizer_val', type=str, default="SGD", help='....')
parser.add_argument('--model_type', type=str, default="UNPRUNED", help='....')
parser.add_argument('--device', type=str, default=None, help='....')
parser.add_argument('--model_input', default=torch.ones((1, 3, 224, 224)), help='....')
parser.add_argument('--eval_metric', default="accuracy", help='....')
parser.add_argument('--pruning_seed', type=int, default=23, help='....')
parser.add_argument('--list_pruning', type=list, default = [0.6,0.6,0.53,0.53,0.4,0.4,0.4,0.5,0.5,0.5,0.6,0.6,0.6,0.5,0.5,0], help='....')

args = parser.parse_args()

if args.device is None:
    import torch
    args.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Get Model, DATASET and TRAIN

In [2]:
#args.pruned_model_name = "VGG16_DISTRI_1"
#args.num_epochs = 40
args.dataset = "Tomato_Leaves"
args.eval_metric = "f1_score"

In [3]:
model = get_model(args)

In [4]:
train_loader, test_loader, num_classes = get_dataset(args)

In [ ]:
train_model(args,
            train_loader = train_loader,
            test_loader = test_loader,
            model = model,
            num_classes = num_classes)

Epoch: [1/40]	 || Training Loss: 0.373	 || Val Loss: 0.154	 || Training F1-score: 0.863 	 ||  Val F1-score: 0.943 	 || Time: 00:07:12
Model Name: VGG16_Tomato_Leaves_UNPRUNED


## Pruning with multiple seeds

In [ ]:
list_seeds = [23,42,97,112,167,214,256,333,425,512]
original_model_name = 'VGG16_Tomato_Leaves_UNPRUNED'
for seed in list_seeds:
    #load original model
    model = torch.load(f'models/{original_model_name}.pth')
    args.seed = seed
    args.model_type = f'50_PRUNED_SEED_{seed}'
    #prune original model
    prune_model(model, args)
    args.model_type = f'50_PRUNED_FT_SEED_{seed}'
    #retraining pruned model
    train_model(args,
                train_loader = train_loader,
                test_loader = test_loader,
                model = model,
                num_classes = num_classes)